# Tutorial: PSA with MDAnalysis and radical.pilot

Analyze an ensemble of trajectories with *Path Similarity Analysis* [Seyler 2015] using [MDAnalysis](http://mdanalysis.org) and [radical.pilot](http://radicalpilot.readthedocs.io/).

## Load packages 

In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis import psa

In [2]:
mda.__version__

'0.16.0-dev0'

In [3]:
import glob
import os

In [9]:
import json

## Trajectories 
The trajectories are an ensemble transitions of the enzyme adenylate kinase (AdK) between closed and open conformation. The trajectories were produced with either DIMS MD or FRODA.

Sean's files:

If you don't want fitted trajectories, look for directories named
"raw", as they should contain trajectories that haven't been
pre-aligned.

In [4]:
DIRECTORIES = {
    "DIMS":
    "/nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/trj/1ake/core",
    "FRODA":
    "/nfs/homes3/sseyler/Simulations/adk/gp/trj/co/fit/core",
}

### DIMS
There are AdK DIMS trajectories (both closed to open and open to
closed) and corresponding topology are, respectively, in:

-   /nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/trj
-   /nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/top

(The subdirectories "core" will have core-aligned trajectories.)


In [17]:
%ls /nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/trj/1ake/core

dims0001_fit-core.dcd  dims0068_fit-core.dcd  dims0135_fit-core.dcd
dims0002_fit-core.dcd  dims0069_fit-core.dcd  dims0136_fit-core.dcd
dims0003_fit-core.dcd  dims0070_fit-core.dcd  dims0137_fit-core.dcd
dims0004_fit-core.dcd  dims0071_fit-core.dcd  dims0138_fit-core.dcd
dims0005_fit-core.dcd  dims0072_fit-core.dcd  dims0139_fit-core.dcd
dims0006_fit-core.dcd  dims0073_fit-core.dcd  dims0140_fit-core.dcd
dims0007_fit-core.dcd  dims0074_fit-core.dcd  dims0141_fit-core.dcd
dims0008_fit-core.dcd  dims0075_fit-core.dcd  dims0142_fit-core.dcd
dims0009_fit-core.dcd  dims0076_fit-core.dcd  dims0143_fit-core.dcd
dims0010_fit-core.dcd  dims0077_fit-core.dcd  dims0144_fit-core.dcd
dims0011_fit-core.dcd  dims0078_fit-core.dcd  dims0145_fit-core.dcd
dims0012_fit-core.dcd  dims0079_fit-core.dcd  dims0146_fit-core.dcd
dims0013_fit-core.dcd  dims0080_fit-core.dcd  dims0147_fit-core.dcd
dims0014_fit-core.dcd  dims0081_fit-core.dcd  dims0148_fit-core.dcd
dims0015_fit-core.dcd  dims0082_fit-core.dcd  di

In [7]:
!du -sh /nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/trj/1ake/core

755M	/nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/trj/1ake/core


Diptheria toxin (DT)

In [16]:
!du -sh /nfs/homes3/sseyler/Simulations/dt/charmm/dims/implicit/trj/co/fit/1ddt

5.0G	/nfs/homes3/sseyler/Simulations/dt/charmm/dims/implicit/trj/co/fit/1ddt


### FRODA

For FRODA, you'll see the relevant top/trj files in:

-   /nfs/homes3/sseyler/Simulations/adk/gp/top
-   /nfs/homes3/sseyler/Simulations/adk/gp/trj/co/fit/core
-   /nfs/homes3/sseyler/Simulations/adk/gp/trj/oc/fit/core

In [8]:
!du -sh /nfs/homes3/sseyler/Simulations/adk/gp/trj/co/fit/core

539M	/nfs/homes3/sseyler/Simulations/adk/gp/trj/co/fit/core


### Load trajectories 

In [5]:
trj_paths = dict((method, glob.glob(os.path.join(pth, "*.dcd"))) 
                 for method, pth in DIRECTORIES.items())
top_paths = {'DIMS': 
             "/nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/top/adk4ake.psf",
             'FRODA': 
             "/nfs/homes3/sseyler/Simulations/adk/gp/top/1ake.pdb"}

### Testing 

In [26]:
dims0 = trj_paths['DIMS'][0]
froda0 = trj_paths['FRODA'][0]

In [32]:
u1 = mda.Universe(top_paths['DIMS'], trj_paths['DIMS'][0])
u2 = mda.Universe(top_paths['FRODA'], trj_paths['FRODA'][0])

In [37]:
ca1 = u1.select_atoms("name CA")
ca2 = u2.select_atoms("name CA")

In [41]:
P = psa.PSAnalysis([u1, u2], ref_select="name CA", targetdir="tmp_psa", labels=["DIMS 0", "FRODA 0"])

In [45]:
P.generate_paths(align=False)

In [49]:
%timeit P.run(metric="discrete_frechet")

1 loop, best of 3: 480 ms per loop


In [52]:
%%timeit 
P = psa.PSAnalysis([u1, u2], ref_select="name CA", targetdir="tmp_psa", labels=["DIMS 0", "FRODA 0"])
P.generate_paths(align=False)
P.run(metric="discrete_frechet")

1 loop, best of 3: 625 ms per loop


In [65]:
import numpy as np

In [69]:
np.savez?

### General approach

1. MAP: split the $N \times M$ matrix into individual pilot jobs that each calculate a sub-matrix $n_i \times m_i$.
2. REDUCE: combine the data into a full matrix

In [56]:
N, M = [len(trj_paths[method]) for method in sorted(trj_paths.keys())]
print(N, M)

(200, 200)


#### Communicating files
Use json to encode a list `[(topol, trj), (topol, trj), ...]`?
Or just command line args?

In [6]:
def make_file_list(method, start=None, stop=None, step=None):
    trj = trj_paths[method][start:stop:step]
    top = len(trj) * [top_paths[method]]

    return top, trj

In [16]:
files = make_file_list('DIMS', stop=20)
f2 = make_file_list('FRODA', stop=20)
files[0].extend(f2[0])
files[1].extend(f2[1])
del f2

In [15]:
print(" ".join(files[0]))
print(" ".join(files[1]))

/nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/top/adk4ake.psf /nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/top/adk4ake.psf /nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/top/adk4ake.psf /nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/top/adk4ake.psf /nfs/homes3/sseyler/Simulations/adk/gp/top/1ake.pdb /nfs/homes3/sseyler/Simulations/adk/gp/top/1ake.pdb /nfs/homes3/sseyler/Simulations/adk/gp/top/1ake.pdb /nfs/homes3/sseyler/Simulations/adk/gp/top/1ake.pdb
/nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/trj/1ake/core/dims0052_fit-core.dcd /nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/trj/1ake/core/dims0001_fit-core.dcd /nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/trj/1ake/core/dims0051_fit-core.dcd /nfs/homes3/sseyler/Projects/Enzymes/AdK/simulations/enhanced/DIMS/trj/1ake/core/dims0002_fit-core.dcd /nfs/homes3/sseyler/Simulations/adk/gp/trj/co/fi

In [17]:
with open("files.json", "w") as inp:
    json.dump(files, inp)